# 词嵌入embedding模型

In [5]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('./app/models/embedding_model',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["What is BGE M3?"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction."]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)
# [[0.6265, 0.3477], [0.3499, 0.678 ]]


[[0.626]]


In [2]:
import re

pattern = r'.*(有限公司|股份)'

# 测试字符串
test_strings = [
    "这是一个测试字符串有限公司",
    "这是另一个测试字符串股份",
    "这是不匹配的字符串"
]

for test_string in test_strings:
    match = re.match(pattern, test_string)
    if match:
        print(f"匹配成功: {match.group()}")
    else:
        print("不匹配")


匹配成功: 这是一个测试字符串有限公司
匹配成功: 这是另一个测试字符串股份
不匹配


In [14]:
import pandas as pd
import json
data1 = pd.read_csv('./app/data/ICL_EXP.csv')
with open('./app/data/lora_data/sql-lora-train.json','r',encoding='utf-8') as file:
    data  = json.load(file)
# data = json.load(open('./app/data/lora_data/sql-lora-train.json', "r"))
new_problem = list(data1['问题'].values)
new_sql = list(data1['SQL'].values)
index = 1001
for p,s in zip(new_problem,new_sql):   
    tmp_data = {
        'id': 'identity_'+str(index),
        'conversations':[{'from': 'user','value': p}, {'from': 'assistant','value': s}]}
    index+=1
    data.append(tmp_data)
with open('./app/data/lora_data/sql-lora-train-end.json','w') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [7]:
data

[{'id': 'identity_0',
  'conversations': [{'from': 'user',
    'value': '在20241231，按照申万行业分类的行业划分标准，哪个一级行业的A股公司数量最多？'},
   {'from': 'assistant',
    'value': 'SELECT "一级行业名称", COUNT(DISTINCT "股票代码") AS "公司数量" FROM "A股公司行业划分表" WHERE "交易日期" = \'20241231\' AND "行业划分标准" = \'申万行业分类\' GROUP BY "一级行业名称" ORDER BY "公司数量" DESC LIMIT 1'}]},
 {'id': 'identity_1',
  'conversations': [{'from': 'user',
    'value': '请查询在20210820日期，中信行业分类下钢铁一级行业中，当日收盘价波动最大（即最高价与最低价之差最大）的股票代码是什么？'},
   {'from': 'assistant',
    'value': "WITH industry_stocks AS (SELECT 股票代码 FROM A股公司行业划分表 WHERE 交易日期 = '20210820' AND 一级行业名称 = '钢铁'), price_fluctuation AS (SELECT 股票代码, ([最高价(元)] - [最低价(元)]) AS 波动 FROM A股票日行情表 WHERE 交易日 = '20210820' AND 股票代码 IN (SELECT 股票代码 FROM industry_stocks)) SELECT 股票代码 FROM price_fluctuation ORDER BY 波动 DESC LIMIT 1;"}]},
 {'id': 'identity_2',
  'conversations': [{'from': 'user',
    'value': '我想知道2014年的年度报告中,机构投资者持有的份额占比超过60%的基金有多少,并且他们总共持有了多少?记得帮我保留两位小数。'},
   {'from': 'assistant',
    'value': "SEL

In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
import dashscope
your_api_key = 'sk-a1a3c4c2b71a498ca2ba23e73ef49348' #这里要填写你自己的api（api开通网址：https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key?spm=a2c4g.11186623.0.0.6c2774fahtfXdn）
dashscope.api_key= your_api_key

# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-turbo',
    'model_server': 'dashscope',
    'api_key': 'sk-a1a3c4c2b71a498ca2ba23e73ef49348',
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    # 'model': 'Qwen2-7B-Chat',
    # 'model_server': 'http://localhost:8000/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',

    # （可选） LLM 的超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
# files = ['./examples/resource/doc.pdf']  # 给智能体一个 PDF 文件阅读。
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                # files=files
                )

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
while True:
    # 例如，输入请求 "绘制一只狗并将其旋转 90 度"。
    query = input('用户请求: ')
    # 将用户请求添加到聊天历史。
    messages.append({'role': 'user', 'content': query})
    response = []
    for response in bot.run(messages=messages):
        # 流式输出。
        print('机器人回应:')
        pprint.pprint(response, indent=2)
    # 将机器人的回应添加到聊天历史。
    messages.extend(response)

机器人回应:
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人回应:
[ { 'content': '',
    'function_call': {'arguments': '{"prompt": "', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人回应:
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt": "一只被旋转90度的狗',
                       'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人回应:
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt": "一只被旋转90度的狗"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人回应:
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt": "一只被旋转90度的狗"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E8%A2%AB%E6%97%8B%E8%BD%AC90%E5%BA%A6%E7%9A%84%E7%8B%97"}',
    'name': 'my_image_gen',
    'role': 'function'}]
机器人回应:
[ { 'content': '',
    'function_call': { '

In [2]:
from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI


def test():
    bot = Assistant(llm={'model': 'qwen-plus'})
    messages = [{'role': 'user', 'content': [{'text': '介绍图一'}, {'file': 'https://arxiv.org/pdf/1706.03762.pdf'}]}]
    for rsp in bot.run(messages):
        print(rsp)


def app_gui():
    # Define the agent
    bot = Assistant(llm={'model': 'qwen-plus'},
                    name='Assistant',
                    description='使用RAG检索并回答，支持文件类型：PDF/Word/PPT/TXT/HTML。')
    chatbot_config = {
        'prompt.suggestions': [
            {
                'text': '介绍图一'
            },
            {
                'text': '第二章第一句话是什么？'
            },
        ]
    }
    WebUI(bot, chatbot_config=chatbot_config).run()


if __name__ == '__main__':
    # test()
    app_gui()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.20.0, however version 4.29.0 is available, please upgrade.
--------


Traceback (most recent call last):
  File "d:\python\lib\site-packages\gradio\queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "d:\python\lib\site-packages\gradio\route_utils.py", line 252, in call_process_api
    output = await app.get_blocks().process_api(
  File "d:\python\lib\site-packages\gradio\blocks.py", line 1664, in process_api
    result = await self.call_function(
  File "d:\python\lib\site-packages\gradio\blocks.py", line 1217, in call_function
    prediction = await utils.async_iteration(iterator)
  File "d:\python\lib\site-packages\gradio\utils.py", line 514, in async_iteration
    return await iterator.__anext__()
  File "d:\python\lib\site-packages\gradio\utils.py", line 507, in __anext__
    return await anyio.to_thread.run_sync(
  File "d:\python\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "d:\python\lib\site-packages\anyio\_backend